In [616]:
import numpy as np
import scipy.sparse as sp
from scipy.linalg import expm
from functools import reduce


def bonds_from_perms(perms):
    """
    Each row p in perms encodes pairs:
    (p[0], p[1]), (p[2], p[3]), ...
    """
    bonds = []
    for p in perms:
        assert len(p) % 2 == 0
        for k in range(0, len(p), 2):
            bonds.append((p[k], p[k+1]))
    return bonds

sx = sp.csr_matrix([[0, 1],
                    [1, 0]], dtype=complex)

sy = sp.csr_matrix([[0, -1j],
                    [1j,  0]], dtype=complex)

sz = sp.csr_matrix([[1,  0],
                    [0, -1]], dtype=complex)

id2 = sp.identity(2, dtype=complex, format='csr')


def two_site_pauli_term(L, i, j, pauli1, pauli2):
    """
    Build the operator:
        I ⊗ ... ⊗ pauli1(at i) ⊗ ... ⊗ pauli2(at j) ⊗ ... I
    """
    if i == j:
        raise ValueError("i and j must be different")

    if i > j:
        i, j = j, i
        pauli1, pauli2 = pauli2, pauli1

    ops = []
    for site in range(L):
        if site == i:
            ops.append(pauli1)
        elif site == j:
            ops.append(pauli2)
        else:
            ops.append(id2)

    op = ops[0]
    for k in range(1, L):
        op = sp.kron(op, ops[k], format='csr')
    return op


def build_H(L, bonds, J, h, n_neighbours):
    dim = 2**L
    H = sp.csr_matrix((dim, dim), dtype=complex)
    for (i, j) in bonds:
        if J[0] != 0:
            H += J[0] * two_site_pauli_term(L, i, j, sx, sx)
        if J[1] != 0:
            H += J[1] * two_site_pauli_term(L, i, j, sy, sy)
        if J[2] != 0:
            H += J[2] * two_site_pauli_term(L, i, j, sz, sz)

        if h[0] != 0:
            H += h[0]  * (two_site_pauli_term(L, i, j, sx, id2) + two_site_pauli_term(L, i, j, id2, sx))/n_neighbours
        if h[1] != 0:
            H += h[1]  * (two_site_pauli_term(L, i, j, sy, id2) + two_site_pauli_term(L, i, j, id2, sy))/n_neighbours
        if h[2] != 0:
            H += h[2]  * (two_site_pauli_term(L, i, j, sz, id2) + two_site_pauli_term(L, i, j, id2, sz))/n_neighbours
    return H

perms_1 = [[0, 4, 6, 10, 2, 5, 8, 11], [4, 6, 10, 0, 5, 8, 11, 2]]
perms_2 = [[0, 1, 2, 3, 6, 7, 8, 9], [1, 2, 3, 0, 7, 8, 9, 6]]
perms_3 = [[1, 4, 9, 11, 3, 5, 7, 10], [4, 1, 11, 9, 5, 7, 10, 3]]
bonds_1 = bonds_from_perms(perms_1)
bonds_2 = bonds_from_perms(perms_2)
bonds_3 = bonds_from_perms(perms_3)
all_bonds = bonds_1 + bonds_2 + bonds_3
L = 12
J = (0, 0, 1)
h = (3, 0, 0)
hamil = build_H(L, all_bonds, J, h, 4)

In [525]:
import sys
import scipy

t = 0.125
sys.path.append("../../src/brickwall_sparse")
from utils_sparse import construct_ising_local_term, reduce_list, X, I2, get_perms, construct_heisenberg_local_term
from ansatz_sparse import ansatz_sparse
import rqcopt as oc
from scipy.sparse.linalg import expm_multiply
from qiskit.quantum_info import random_statevector
from scipy.linalg import expm
from qiskit.quantum_info import state_fidelity

X = np.array([[0, 1], [1, 0]])
Z = np.array([[1, 0], [0, -1]])
Y = np.array([[0, -1j], [1j, 0]])
YZ = np.kron(Y, Z)
I2 = np.array([[1, 0], [0, 1]])

state = np.array(random_statevector(2**L).data)
hloc = construct_heisenberg_local_term((0, 0,    J[2]), (h[0], 0,    0), ndim=2)

V = scipy.linalg.expm(-1j*t*hloc)
YZ = np.kron(Y, Z)
Vlist_start =  [YZ, V, YZ, YZ, V, YZ, YZ, V, YZ]
Vlist_reduced = [V, V, V]
perms_extended = [[perms_1[0]]] + [perms_1] + [[perms_1[0]], [perms_2[0]]] +\
                    [perms_2] + [[perms_2[0]], [perms_3[0]]] + [perms_3] + [[perms_3[0]]] 
perms_ext_reduced = [perms_1]  + [perms_2] + [perms_3]
control_layers = [0, 2, 3, 5, 6, 8]



print("Trotter error of the starting point: ", 1-state_fidelity(ansatz_sparse(Vlist_start, L, perms_extended, state), expm_multiply(
    1j * t * hamil, state)))
print("Trotter error of the starting point: ", 1-state_fidelity(ansatz_sparse(Vlist_reduced, L, perms_ext_reduced, state), expm_multiply(
    -1j * t * hamil, state)))
print("Trotter error of the starting point: ", (np.linalg.norm(ansatz_sparse(Vlist_start, L, perms_extended, state) - expm_multiply(
    1j * t * hamil, state), ord=2) + np.linalg.norm(ansatz_sparse(Vlist_reduced, L, perms_ext_reduced, state) - expm_multiply(
    -1j * t * hamil, state), ord=2))/2)

Trotter error of the starting point:  0.03290317319493108
Trotter error of the starting point:  0.03340737673091687
Trotter error of the starting point:  0.18285450055700725


In [559]:
Vlist_reduced = [Vlist[1], Vlist[4], Vlist[7]]

In [527]:
from optimize_sparse import optimize
import h5py

niter = 8
Vlist, f_iter, err_iter = optimize(L, hamil, t, Vlist, perms_extended, perms_reduced=perms_ext_reduced,
                                   control_layers=control_layers, rS=1, niter=niter)

with h5py.File(f"./results/kagome_TFIM_L{L}_t{t}_layers{len(Vlist)}.hdf5", "w") as f:
    f.create_dataset("Vlist", data=Vlist)
    f.create_dataset("f_iter", data=f_iter)
    f.create_dataset("err_iter", data=err_iter)
    f.attrs["L"] = L
    f.attrs["t"] = float(t)

Current error:  0.05656376490989869
Current error:  0.0547791886874832
Current error:  0.05404069128671259
Current error:  0.053153338322415936
Current error:  0.0517996443601167
Current error:  0.051074935646549746
Current error:  0.05102848673192702
Current error:  0.051008347374989056
Current error:  0.050997620938936415


In [642]:
1-np.abs(np.trace(Vlist[3].conj().T@YZ))/4

0.0030505872711221738

In [668]:
from scipy import sparse as sp
from utils_sparse import applyG_state

"""
    TODO: Somehow this mapping qiskit -> np isnt working
    Maybe do it with applyG and count gates manually
"""

sv = random_statevector(2**L).data
state1 = np.kron(ket_0, sv)
state2 = np.kron(ket_1, sv)


def apply(state):
    count = 0
    for i, V in enumerate(Vlist):
        layer = i
        if i in control_layers:
            Glist = Xlists_opt[i]
            for perm in perms_extended[i]:
                for j in range(len(perm)//2):
                    for _, G in enumerate(Glist):
                        
                        mapping = {0: 0, 1: perm[2*j]+1, 2: perm[2*j+1]+1}
                        state = applyG_state(G, state, L+1, mapping[perms_qc[_][0]], mapping[perms_qc[_][1]])
                        count += 1
                
        else:
            for perm in perms_extended[layer]:
                for j in range(len(perm)//2):
                    state = applyG_state(V, state, L+1, perm[2*j]+1, perm[2*j+1]+1)
                    count += 2
    return state, count

sv1, count = apply(state1)
sv2, count1 = apply(state2)

exact_v1 = np.kron(ket_0, expm_multiply(-1j * t * hamil, sv))
exact_v2 = np.kron(ket_1, expm_multiply(1j * t * hamil, sv))
print((1-state_fidelity(sv1, exact_v1) + 1-state_fidelity(sv2, exact_v2))/2)

count

0.008395094948197512


96

In [619]:
state = random_statevector(2**L).data
perms_extended = [[perms_1[0]]] + [perms_1] + [[perms_1[0]], [perms_2[0]]] +\
                    [perms_2] + [[perms_2[0]], [perms_3[0]]] + [perms_3] + [[perms_3[0]]] 
perms_ext_reduced = [perms_1]  + [perms_2] + [perms_3]
control_layers = [0, 2, 3, 5, 6, 8]

print("Trotter error of the starting point: ", 1-state_fidelity(ansatz_sparse(Vlist_start, L, perms_extended, state), expm_multiply(
    1j * t * hamil, state)))
print("Trotter error of the optimized point: ", 1-state_fidelity(ansatz_sparse(Vlist, L, perms_extended, state), expm_multiply(
    1j * t * hamil, state)))
print("Trotter error of the optimized point: ", 1-state_fidelity(ansatz_sparse(Vlist_reduced, L, perms_ext_reduced, state), expm_multiply(
    -1j * t * hamil, state)))

Trotter error of the starting point:  0.03328163928851957
Trotter error of the optimized point:  0.0008792940316135756
Trotter error of the optimized point:  0.0055832608608638035


In [648]:
sys.path.append("../../src/controlled_unitary_optimizer")
sys.path.append("../../src/brickwall_ansatz")
from optimize_3q import optimize_3q 
from utils_3q import make_controlled, random_unitary

Xlists_opt = {}
#perms_qc = [[0, 1], [0, 2], [1, 2], [0, 2], [0, 1], [1, 2], [0, 2], [0, 1], [1, 2], [1, 2], [0, 1], [0, 2]]
perms_qc = [[0, 1], [0, 2]]

for i in control_layers:
    cU = make_controlled(Vlist[i])
    f_best, err_best, Glist_best = (0, 2, None)
    for _ in range(3):
        Xlist_start = [random_unitary(4) for i in range(len(perms_qc))]
        Xlist, f_iter, err_iter = optimize_3q(L, cU, Xlist_start, perms_qc, niter=1000)
        if err_iter[-1] < err_best:
            f_best, err_best, Xlist_best = (f_iter[-1], err_iter[-1], Xlist)
    print("Best f: ", f_best)
    print("Best err: ", err_best)
    Xlists_opt[i] = Xlist_best

Best f:  -7.997335067749082
Best err:  0.043072540679392035
Best f:  -7.9960145490121235
Best err:  0.047422968707125784
Best f:  -7.991867420619622
Best err:  0.07075146468588212
Best f:  -7.989858330512304
Best err:  0.07243093074871985
Best f:  -7.994931805551152
Best err:  0.054706417514804304
Best f:  -7.993406911607346
Best err:  0.06075363325790433


In [571]:
import qiskit
from qiskit import Aer, execute, transpile
from qiskit.circuit.library import CYGate, CZGate, IGate, CXGate
from qiskit.converters import circuit_to_dag
from qiskit.providers.aer.noise import NoiseModel, errors
from qiskit import Aer, execute, transpile
from scipy import sparse as sp

"""qc = qiskit.QuantumCircuit(L+1)
qc.x(L)
for i, V in enumerate(Vlist):
    layer = i
    if i in control_layers:
        G = make_controlled(Vlist[i])
        #Glist = Xlists_opt[i]
        qc_3 = qiskit.QuantumCircuit(3)
        #for j, G in enumerate(Glist):
        #    qc_3.unitary( G, (3-1-perms_qc[j][1], 3-1-perms_qc[j][0]))
        qc_3.unitary( G, [0, 1, 2])
        
        for perm in perms_extended[layer]:
            for j in range(len(perm)//2):
                qc.append(qc_3.to_gate(), [L-perm[2*j]-1, L-perm[2*j+1]-1, L])
        
    else:
        for perm in perms_extended[layer]:
            for j in range(len(perm)//2):
                qc.unitary(V, [L-perm[2*j]-1, L-perm[2*j+1]-1])
qc.x(L)"""


ccU_cxs= []
ccU_errs = []
for t_ in [0.125]:
    state = random_statevector(2**L).data
    qc_ext1 = qiskit.QuantumCircuit(L+1)
    qc_ext1.initialize(state, [i for i in range(L)])
    for i in range(int(t_/t)):
        qc_ext1.append(qc.to_gate(), [i for i in range(L+1)])
    backend = Aer.get_backend("statevector_simulator")
    sv1 = execute(transpile(qc_ext1), backend).result().get_statevector().data
    #sv1 = np.kron(ket_0, ansatz_sparse(Vlist, L, perms_extended, state))
    
    qc_ext2 = qiskit.QuantumCircuit(L+1)
    qc_ext2.initialize(state, [i for i in range(L)])
    qc_ext2.x(L)
    for i in range(int(t_/t)):
        qc_ext2.append(qc.to_gate(), [i for i in range(L+1)])
    backend = Aer.get_backend("statevector_simulator")
    sv2 = execute(transpile(qc_ext2), backend).result().get_statevector().data
    #sv2 = np.kron(ket_1, ansatz_sparse(Vlist_reduced, L, perms_ext_reduced, state))

    ket_0 = np.array([1, 0])
    ket_1 = np.array([0, 1])
    exact_v1 = np.kron(ket_0, expm_multiply(1j * t_ * hamil, state))
    exact_v2 = np.kron(ket_1, expm_multiply(-1j * t_ * hamil, state))
    err = (np.linalg.norm(sv1-exact_v1, ord=2) + np.linalg.norm(sv2-exact_v2, ord=2))/2
    print((1-state_fidelity(sv1, exact_v1) + 1-state_fidelity(sv2, exact_v2))/2)
    
    noise_model = NoiseModel()
    dag = circuit_to_dag(transpile(qc_ext1, basis_gates=noise_model.basis_gates+['unitary', 'initialize', 'cx']))
    count_ops = dag.count_ops()
    
    ccU_errs.append(err)
    ccU_cxs.append(count_ops['unitary'])
    #print(f"t={t_}, Gate Count: ", count_ops['unitary'], " Trotter error: ", err)

0.0753136656590025


In [692]:
"""
    Now here is to compare the performance of the ccU circuit
    with the 1st and 2nd order Trotter circuits, in terms of 
    gate count vs Trotter error. I demonstrate it on L=10 system.
"""

from qiskit import Aer, execute, transpile
from qiskit.circuit.library import CYGate, CZGate, IGate, CXGate
from qiskit.converters import circuit_to_dag
from qiskit.providers.aer.noise import NoiseModel, errors
from qiskit import Aer, execute, transpile
from scipy import sparse as sp


def controlled_trotter(t, L, Lx, Ly, J, h, perms_1, perms_2, perms_3, dag=False, nsteps=1, trotter_order=2):
    t = t/nsteps
    print("nsteps ", nsteps)

    hloc1 = construct_heisenberg_local_term((0, 0   ,    J[2]), (0, 0,       0), ndim=2)
    hloc2 = construct_heisenberg_local_term((0   ,    0, 0), (h[0], 0, 0      ), ndim=2)
    #hloc3 = construct_heisenberg_local_term((0   , 0   , J[2]), (h[0], 0,       0), ndim=2)
    hlocs = (hloc1,hloc2, )

    # Suzuki splitting
    if trotter_order > 1:
        sm = oc.SplittingMethod.suzuki(len(hlocs), int(np.log(trotter_order)/np.log(2)))
        indices, coeffs = sm.indices, sm.coeffs
    else:
        indices, coeffs = range(len(hlocs)), [1]*len(hlocs)
    perms_ext = [perms_1, perms_2, perms_3]*len(indices)
    
    cgates = ((CYGate, None),
              (CYGate, CYGate),
             )
    
    K = []
    for i, perms in enumerate(perms_ext):
        sub = int(i//3)
        index = indices[sub]
        perm = perms[0]
        K_layer = [None for _ in range(L)]
        for j in range(len(perm)//2):
            K_layer[perm[2*j]] =  cgates[index][0]
            K_layer[perm[2*j+1]] =  cgates[index][1]
        K.append(K_layer)

    Vlist_start = []
    for i, c in zip(indices, coeffs):
        Vlist_start.append(scipy.linalg.expm(-1j*c*t*hlocs[i]))
    Vlist_gates = []
    for V in Vlist_start:
        qc2 = qiskit.QuantumCircuit(2)
        qc2.unitary(V, [0, 1], label='str')
        Vlist_gates.append(qc2)
    
    qc = qiskit.QuantumCircuit(L+1)
    for n in range(nsteps):
        for layer, qc_gate in enumerate(Vlist_gates):
            for _, perms in enumerate([perms_1, perms_2, perms_3]):
                qc.x(L)
                for j in range(L):
                    if K[3*layer+_][j]:
                        qc.append(K[3*layer+_][j](), [L, L-1-j])
                qc.x(L)
                
                for perm in perms:
                    for j in range(len(perm)//2):
                        qc.append(qc_gate.to_gate(), [L-(perm[2*j]+1), L-(perm[2*j+1]+1)])
                        
                qc.x(L)
                for j in range(L):
                    if K[3*layer+_][j]:
                        qc.append(K[3*layer+_][j](), [L, L-1-j])
                qc.x(L)

    return qc

trotter1_cxs_01 = []
trotter1_errs_01 = []
for t_ in [t]:
    state = random_statevector(2**L).data
    qc_ext1 = qiskit.QuantumCircuit(L+1)
    qc_ext1.initialize(state, [i for i in range(L)])
    qc_ext1.append(controlled_trotter(t_, L, Lx, Ly, J, h, perms_1, perms_2, perms_3).to_gate(), [i for i in range(L+1)])
    backend = Aer.get_backend("statevector_simulator")
    sv1_T = execute(transpile(qc_ext1), backend).result().get_statevector().data
    
    qc_ext2 = qiskit.QuantumCircuit(L+1)
    qc_ext2.initialize(state, [i for i in range(L)])
    qc_ext2.x(L)
    qc_ext2.append(controlled_trotter(t_, L, Lx, Ly, J, h, perms_1, perms_2, perms_3).to_gate(), [i for i in range(L+1)])
    backend = Aer.get_backend("statevector_simulator")
    sv2_T = execute(transpile(qc_ext2), backend).result().get_statevector().data

    ket_0 = np.array([1, 0])
    ket_1 = np.array([0, 1])
    exact_v1 = np.kron(ket_0, expm_multiply(1j * t_ * hamil, state))
    exact_v2 = np.kron(ket_1, expm_multiply(-1j * t_ * hamil, state))
    
    #err = (np.linalg.norm(sv1_T-exact_v1, ord=2) + np.linalg.norm(sv2_T-exact_v2, ord=2))/2
    err1 = 1-state_fidelity(sv1_T, exact_v1)
    err2 = 1-state_fidelity(sv2_T, exact_v2)
    
    #err1 = np.linalg.norm(sv1_T - exact_v1, ord=2) 
    #err2 = np.linalg.norm(sv2_T - exact_v2, ord=2)

    noise_model = NoiseModel()
    dag = circuit_to_dag(transpile(qc_ext1, optimization_level=2, basis_gates=noise_model.basis_gates+['initialize', 'cx', 'u3']))
    count_ops = dag.count_ops()

    trotter1_cxs_01.append(count_ops['cx'])
    trotter1_errs_01.append(err)

    print(f"t={t}, Gate count: ", count_ops, " SV error: ", (err1+err2)/2)

nsteps  1
nsteps  1
t=0.125, Gate count:  {'initialize': 1, 'u3': 222, 'rz': 12, 'cx': 128}  SV error:  0.007211448953165389


In [ ]:
# Trotter: 128 -> 3312 (N=108)
# TICC:    96 ->  864  (N=108)